In [33]:
!pip install spacy
import numpy as np
import spacy
import re
import unicodedata
!python -m spacy download en_core_web_lg
# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

     |████████████████████████████████| 827.9 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [34]:
!pip install texthero


In [35]:
import pandas as pd

# Loading data
comments = pd.read_csv('train.csv.zip')
#comments = comments.head(10)
comments = comments.drop(comments[(comments['toxic']+comments['severe_toxic']+comments['obscene']+comments['threat']+comments['insult']+comments['identity_hate']) == 0].index)
comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1,1,1,0,1,0
12,0005c987bdfc9d4b,Hey... what is it..\n@ | talk .\nWhat is it......,1,0,0,0,0,0
16,0007e25b2121310b,"Bye! \n\nDon't look, come or think of comming ...",1,0,0,0,0,0
42,001810bf8c45bf5f,You are gay or antisemmitian? \n\nArchangel WH...,1,0,1,0,1,1
43,00190820581d90ce,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1,0,1,0,1,0


In [36]:
#create a custom cleaning pipeline
custom_pipeline = [preprocessing.fillna
                   , preprocessing.lowercase
                   , preprocessing.remove_digits
                   , preprocessing.remove_punctuation
                   , preprocessing.remove_diacritics
                   #, preprocessing.remove_stopwords
                   , preprocessing.remove_whitespace
                   , preprocessing.stem]
#pass the custom_pipeline to the pipeline argument
comments['comment_text'] = hero.clean(comments['comment_text'], pipeline = custom_pipeline)
comments.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
6,0002bcb3da6cb337,cocksuck befor you piss around on my work,1,1,1,0,1,0
12,0005c987bdfc9d4b,hey what is it talk what is it an exclus group...,1,0,0,0,0,0
16,0007e25b2121310b,bye don t look come or think of com back tosser,1,0,0,0,0,0
42,001810bf8c45bf5f,you are gay or antisemmitian archangel white t...,1,0,1,0,1,1
43,00190820581d90ce,fuck your filthi mother in the ass dri,1,0,1,0,1,0


In [37]:
#comments_sample = comments[:1000]
comments_sample = comments
#print(comments.shape)
with nlp.disable_pipes():
    doc_vectors = np.array([nlp(text).vector for text in comments_sample.comment_text])
np.savetxt("toxic_clean_vectors.csv", doc_vectors, delimiter=",")
doc_vectors.shape

(16225, 300)

In [39]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X_train, X_test, y_train, y_test = train_test_split(doc_vectors, comments_sample.toxic,
                                                    test_size=0.2, random_state=1)

In [40]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

# Set dual=False to speed up training, and it's not needed
svc_toxic = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc_toxic.fit(X_train, y_train)

LinearSVC(dual=False, max_iter=10000, random_state=1)

In [41]:
print(f"roc_auc_score: {roc_auc_score( y_test, svc_toxic.predict(X_test)) * 100:.3f}%", )

roc_auc_score: 50.000%


In [ ]:
#import pickle
#pickle.dump(svc, open('model_svc', 'wb'))

In [ ]:
#loaded_model = pickle.load(open('model_svc', 'rb'))
#result = loaded_model.score(X_test, y_test)

In [ ]:
#print(result)

0.955


In [42]:
#For Severe Toxic
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, comments_sample.severe_toxic,
                                                    test_size=0.2, random_state=1)

In [43]:
svc_severe = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc_severe.fit(X_train, y_train)
print(f"roc_auc_score: {roc_auc_score( y_test, svc_severe.predict(X_test)) * 100:.3f}%", )

roc_auc_score: 57.861%


In [44]:
#For obscene
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, comments_sample.obscene,
                                                    test_size=0.2, random_state=1)

In [45]:
svc_obscene = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc_obscene.fit(X_train, y_train)

LinearSVC(dual=False, max_iter=10000, random_state=1)

In [46]:
print(f"roc_auc_score: {roc_auc_score( y_test, svc_obscene.predict(X_test)) * 100:.3f}%", )

roc_auc_score: 77.539%


In [47]:
#For Threat
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, comments_sample.threat,
                                                    test_size=0.2, random_state=1)

In [48]:
svc_threat = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc_threat.fit(X_train, y_train)

LinearSVC(dual=False, max_iter=10000, random_state=1)

In [49]:
print(f"roc_auc_score: {roc_auc_score( y_test, svc_threat.predict(X_test)) * 100:.3f}%", )

roc_auc_score: 55.464%


In [50]:
#For Insult
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, comments_sample.insult,
                                                    test_size=0.2, random_state=1)

In [51]:
svc_insult = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc_insult.fit(X_train, y_train)

LinearSVC(dual=False, max_iter=10000, random_state=1)

In [52]:
print(f"roc_auc_score: {roc_auc_score( y_test, svc_insult.predict(X_test)) * 100:.3f}%", )

roc_auc_score: 68.617%


In [53]:
#For identity_hate
X_train, X_test, y_train, y_test = train_test_split(doc_vectors, comments_sample.identity_hate,
                                                    test_size=0.2, random_state=1)

In [54]:
svc_identity_hate = LinearSVC(random_state=1, dual=False, max_iter=10000)
svc_identity_hate.fit(X_train, y_train)

LinearSVC(dual=False, max_iter=10000, random_state=1)

In [55]:
print(f"roc_auc_score: {roc_auc_score( y_test, svc_identity_hate.predict(X_test)) * 100:.3f}%", )

roc_auc_score: 55.511%


In [56]:
test_comments = pd.read_csv('test.csv.zip')
test_comments.head()


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [60]:
test_comments_sample = test_comments.head(1000)
test_comments_sample['comment_text'] = hero.clean(test_comments_sample['comment_text'], pipeline = custom_pipeline)
with nlp.disable_pipes():
    test_doc_vectors = np.array([nlp(text).vector for text in test_comments_sample.comment_text])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [63]:
predictions = pd.DataFrame()
predictions["id"] = test_comments_sample.id
predictions["toxic"] = svc_toxic.predict(test_doc_vectors)
predictions["severe"] = svc_severe.predict(test_doc_vectors)
predictions["obscene"] = svc_obscene.predict(test_doc_vectors)
predictions["threat"] = svc_threat.predict(test_doc_vectors)
predictions["insult"] = svc_insult.predict(test_doc_vectors)
predictions["identity_hate"] = svc_identity_hate.predict(test_doc_vectors)



In [64]:
predictions.head()

,id,toxic,severe,obscene,threat,insult,identity_hate
0,00001cee341fdb12,1,0,1,0,1,0
1,0000247867823ef7,1,0,1,0,0,0
2,00013b17ad220c46,1,0,1,0,0,0
3,00017563c3f7919a,1,0,0,0,0,0
4,00017695ad8997eb,1,0,0,0,0,0


In [65]:
predictions.to_csv('submission.csv',index=False)